# 環境情報判断

走行の推論と環境分類の推論の同時起動は、Jetson NX Xavier, Jetson Orin Nanoでの対応となります。Jetson Nanoでは動作が不安定になります。

走行中の画像から分類します。スタート位置とそれ以外の分類や、カーブと直線の分類など可能になります。

### Boardを判定

In [ ]:
import Jetson.GPIO as GPIO

BOARD_NAME = GPIO.gpio_pin_data.get_data()[0]

mode_descriptions = {
    "JETSON_NX": ["15W_2CORE", "15W_4CORE", "15W_6CORE", "10W_2CORE", "10W_4CORE"],
    "JETSON_XAVIER": ["MAXN", "MODE_10W", "MODE_15W", "MODE_30W"],
    "JETSON_NANO": ["MAXN", "5W"],
    "JETSON_ORIN": ["MAXN", "MODE_15W", "MODE_30W", "MODE_40W"],
    "JETSON_ORIN_NANO": ["MODE_15W", "MODE_7W"]
}

product_names = {
    "JETSON_NX": "Jetson Xavier NX",
    "JETSON_XAVIER": "Jetson AGX Xavier",
    "JETSON_NANO": "Jetson Nano",
    "JETSON_ORIN": "Jetson AGX Orin",
    "JETSON_ORIN_NANO": "Jetson Orin Nano"
}

# ボードごとのI2Cバス番号と初期Powerモードを定義する
board_settings = {
    "JETSON_NX": (8, 3),
    "JETSON_XAVIER": (8, 2),
    "JETSON_NANO": (1, 0),
    "JETSON_ORIN": (7, 0),
    "JETSON_ORIN_NANO": (7, 0)
}

i2c_busnum, power_mode = board_settings.get(BOARD_NAME, (None, None))
mode_description = mode_descriptions.get(BOARD_NAME, [])
product_name = product_names.get(BOARD_NAME, "未知のボード")

if power_mode is not None and power_mode < len(mode_description):
    mode_str = mode_description[power_mode]
    print("------------------------------------------------------------")
    print(f"{product_name}を認識: I2Cバス番号: {i2c_busnum}, Powerモード: {mode_str}({power_mode})に設定します。")
    print("------------------------------------------------------------")
else:
    print("未知のボードまたは不正なモードです。")

In [ ]:
if (product_name == "Jetson Orin Nano") or (product_name == "Jetson AGX Orin"):
    print("Docker起動のため電力モードは変更できません。")
else:
    !echo "jetson" | sudo -S nvpmodel -m $power_mode

In [ ]:
!echo "jetson" | sudo -S nvpmodel -q

In [ ]:
if (product_name == "Jetson Orin Nano") or (product_name == "Jetson AGX Orin"):
    print("Docker起動のためjetson_clocksは起動できません。")
else:
    !echo "jetson" | sudo -S jetson_clocks

## dataset.pyの取得

https://github.com/sangyy/jetson-dlinano/tree/master/classification

のページよりdataset.pyをダウンロードし、JetRacerのJupyterlabにアップロードします。下記コマンドでも取得可能です。ダウンロードが完了したら、JupyterLabのNotebookをShutdownし、もう一度開き直します。

In [ ]:
import os
file_path = 'dataset.py'
if not os.path.exists(file_path):
    !wget https://raw.githubusercontent.com/sangyy/jetson-dlinano/master/classification/dataset.py

In [ ]:
import ipywidgets as widgets
from jetcam.utils import bgr8_to_jpeg
import os
from IPython.display import display
from ipywidgets import Button, Layout
from jupyter_clickable_image_widget import ClickableImageWidget
from dataset import ImageClassificationDataset
import torchvision.transforms as transforms

WIDTH = 80
HEIGHT = 80

LOAD_DATASETS = []
LOAD_TASK = ['camera','dataset']

CATEGORIES = ["start","road","etc"]
SAVE_DATASETS = []
SAVE_TASK = ['detect']

current_path = os.getcwd()

try:
    path = os.path.join(current_path, SAVE_TASK[0])
    dirs = os.listdir(path)
    dirs = [f for f in dirs if os.path.isdir(os.path.join(path, f))]
    dirs = [f for f in dirs if f != ".ipynb_checkpoints"]
    dirs = sorted(dirs)
    if len(dirs) > 0:
        SAVE_DATASETS = dirs
    else:
        SAVE_DATASETS = []
except:
    SAVE_DATASETS = []
    
TRANSFORMS = transforms.Compose([
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.2),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

datasets = {}
for name in SAVE_DATASETS:
    datasets[name] = ImageClassificationDataset(SAVE_TASK[0] + '/' + name, CATEGORIES, TRANSFORMS)

try:
    dataset = datasets[SAVE_DATASETS[0]]
except:
    dataset = {}
    
SIZE = 8
no = 0

In [ ]:
# 画像を表示するウィジェット
snapshot_widgets = []
for _ in range(SIZE):
    snapshot_widgets.append(widgets.Image(width=WIDTH, height=HEIGHT))

# 画像枚数
image_count_widget = widgets.IntText(description='画像枚数', value=0)

# 読込関連ウィジェット
load_image_button = widgets.Button(description='読込')
load_datasets_dropdown = widgets.Dropdown(options=LOAD_DATASETS, description='読込dataset')
load_task_dropdown = widgets.Dropdown(options=LOAD_TASK, description='読込task', index=0)

# 保存関連ウィジェット
save_datasets_dropdown = widgets.Dropdown(options=SAVE_DATASETS, description='dataset')
save_task_dropdown = widgets.Dropdown(options=SAVE_TASK, description='保存task', index=0)
dataset_create_button = widgets.Button(description='Create dataset')
dataset_name_widget = widgets.Text(description='Name')

# 画像ナビゲーションボタン
prev_image_button = widgets.Button(description='<')
next_image_button = widgets.Button(description='>')

# 更新ボタン
update_button = widgets.Button(description='更新')

# モデルの読込み, 保存ボタン
load_model_button = widgets.Button(description='load model')
load_model_widget = widgets.Dropdown(options=[],description='読込モデル')
load_model_time_widget = widgets.Text(description='作成日時')
save_model_name_widget = widgets.Text(description='保存モデル名',value="model.pth")
save_model_button = widgets.Button(description='保存')

# 画像の操作
SLEEP = [50,100,200,300,400,500]
SKIP = [1,2,3,4,5]
play_button = widgets.Button(description='▶')
stop_button = widgets.Button(description='⏹')
next_image_button = widgets.Button(description='>')
prev_image_button = widgets.Button(description='<')
picture_widget = ClickableImageWidget(width=224, height=224)
no_widget = widgets.IntText(description='no')
update_image_button = widgets.Button(description='更新')
delete_image_button = widgets.Button(description='削除')
sleep_dropdown = widgets.Dropdown(options=SLEEP, description='sleep(ms)', index=0)
skip_dropdown = widgets.Dropdown(options=SKIP, description='skip(枚)', index=1)

In [ ]:
l = Layout(flex='0 1 auto', height='100px', min_height='100px', width='auto')
process_widget = widgets.Textarea(description='ログ', value='', layout=l)

process_no = 0
def write_log(msg):
    global process_widget, process_no
    process_no = process_no + 1
    process_widget.value = str(process_no) + ": " + msg + "\n" + process_widget.value

In [ ]:
from packaging import version
import torch
import torchvision

torchvision_version = version.parse(torchvision.__version__)

device = torch.device('cuda')
output_dim = len(CATEGORIES)  

if torchvision_version >= version.parse("0.13"):
    print(torchvision_version)
    from torchvision.models.resnet import ResNet18_Weights, resnet18
    weights = ResNet18_Weights.DEFAULT
    model = resnet18(weights=weights)
    model.fc = torch.nn.Linear(model.fc.in_features, output_dim)
else:
    model = torchvision.models.resnet18(pretrained=True)
    model.fc = torch.nn.Linear(512, output_dim)

model = model.to(device)

In [ ]:
import glob

def load_model(change):
    global torchvision_version, model, device, output_dim, torch
    
    model_name = load_model_widget.value
    device = torch.device('cuda')
    if torchvision_version >= version.parse("0.13"):
        from torchvision.models.resnet import ResNet18_Weights, resnet18
        if model_name == "[new]":
            weights = ResNet18_Weights.DEFAULT
            model = resnet18(weights=weights)
            model.fc = torch.nn.Linear(model.fc.in_features, output_dim)
            write_log(f"[new]が選択されたのでresnet18の最新の重みから始めます。")
        else:
            model = resnet18(weights=None)  
            model.fc = torch.nn.Linear(model.fc.in_features, output_dim)
            model.load_state_dict(torch.load(model_name))
            model.eval()
            write_log(model_name + "のモデルを読込ました。")            
    else:
        if model_name == "[new]":
            model = torchvision.models.resnet18(pretrained=True)
            model.fc = torch.nn.Linear(512, output_dim)
            write_log("[new]が選択されたのでresnet18のpretrainedから始めます。")
        else:
            model = torchvision.models.resnet18(pretrained=False)
            model.fc = torch.nn.Linear(512, output_dim)
            model.load_state_dict(torch.load(model_name))
            model.eval()
            write_log(model_name + "のモデルを読込ました。")
            
    model = model.to(device)
    #get_jetson_nano_memory_usage()

def save_model(change):
    global model, device, current_path
    path = os.path.join(current_path, "model_c")
    if not os.path.exists(path):
        subprocess.call(['mkdir', '-p', path])
    torch.save(model.state_dict(), path + "/" + save_model_name_widget.value)
    write_log(path + "/" + save_model_name_widget.value + "に保存しました。")
    
def model_list(change):
    try:
        files = glob.glob('./model_c/*.pth', recursive=True)
        files.insert(0,"[new]")
        load_model_widget.options = files
        load_model_time_widget.value = ""
    except Exception as e:
        write_log(f"{e}")
        load_model_widget.options = ["[new]"]
        
    
model_list("list")
load_model_button.on_click(load_model)

In [ ]:
from functools import partial

score_widgets = []

def add_image(index, b):
    global load_flag, no, img, dataset, CATEGORIES
    dataset.save_entry(img, CATEGORIES[index])
    score_widgets[index].children[1].value = dataset.get_count(CATEGORIES[index])
    write_log(f"{CATEGORIES[index]}に追加")

for i, category in enumerate(CATEGORIES):
    score_widget = widgets.FloatSlider(min=0.0, max=1.0, description=category, orientation='vertical')
    number_widget = widgets.IntText(description='', value=0, layout=widgets.Layout(width='100px'))
    add_button = widgets.Button(description="追加", layout=widgets.Layout(width=f'100px', height=f'30px'))
    add_button.on_click(partial(add_image, i))
    
    score_widgets.append(widgets.VBox([score_widget,number_widget,add_button]))

In [ ]:
import os
import cv2
import numpy as np
import shutil
import re

def extract_numbers(filename):
    matches = re.findall(r'(\d+)', filename)
    if matches and len(matches) >= 3: 
        return int(matches[-1])  
    else:
        return float('inf')

def get_file_names(path):
    file_names = os.listdir(path)
    file_names = [os.path.join(path, file_name) for file_name in file_names]
    image_names = []

    image_names = sorted(file_names, key=lambda f: extract_numbers(os.path.basename(f)))
    image_names = [f for f in image_names if os.path.splitext(f)[1].lower() == ".jpg"]
    
    return image_names

def next_image(c):
    global no, load_flag, last_no
    next_image_button.disabled = True
    prev_image_button.disabled = True
    load_flag = True
    no += 1
    if no < last_no:
        no_widget.value = no
        load_image(no)
    else:
        no -= 1
        next_image_button.disabled = False
        prev_image_button.disabled = False
    
def prev_image(c):
    global no, load_flag, last_no
    next_image_button.disabled = True
    prev_image_button.disabled = True
    load_flag = True
    no -= 1
    if no < 0:
        no = 0
    no_widget.value = no
    load_image(no)
    
def update_image(c):
    global no, last_no
    if no < 0:
        no = 0
    elif no < last_no:
        start_number = no
        end_number = no + SIZE
        load_image(no)
    
def change_load_task(c):
    global current_path
    try:
        path = os.path.join(current_path, load_task_dropdown.value)     
        files = os.listdir(path)
        dirs = [f for f in files if os.path.isdir(os.path.join(path, f))]
        dirs = [f for f in files if f != ".ipynb_checkpoints"]
        dirs = sorted(dirs)
        load_datasets_dropdown.options = dirs
    except:
        write_log(path + "が存在していません。")
        load_datasets_dropdown.options = []
    
def load_dataset(change):
    global load_flag, no, play_num
    no = 0
    play_num = 0
    write_log(f"{load_task_dropdown.value}/{load_datasets_dropdown.value}を呼び込み(30秒〜1分以上かかります)")
    load_image(0)
    
def update_image(change):
    global load_flag, no
    load_flag = True
    no = no_widget.value
    load_image(no)

def create_dataset(c):
    global dataset,datasets,score_widgets,CATEGORIES,TRANSFORMS,SAVE_DATASETS
    new_dataset_name = dataset_name_widget.value
    write_log(f"{new_dataset_name}を生成します。")
    if new_dataset_name not in SAVE_DATASETS:
        SAVE_DATASETS.append(new_dataset_name)

    datasets = {}
    for name in SAVE_DATASETS:
        write_log(f"name: {name}")
        datasets[name] = ImageClassificationDataset(save_task_dropdown.value + '/' + name, CATEGORIES, TRANSFORMS)
         
    dataset = datasets[new_dataset_name]  

    for i,score_widget in enumerate(score_widgets):
        score_widget.children[1].value = 0
        
    write_log("Datasetを作成しました：" + new_dataset_name)
    save_datasets_dropdown.options = SAVE_DATASETS
    save_datasets_dropdown.value = new_dataset_name
   
def change_save_task(change):
    global current_path
    try:
        path = os.path.join(current_path, save_task_dropdown.value)
        if not os.path.exists(path):
            subprocess.call(['mkdir', '-p', path])
        files = os.listdir(path)            
        dirs = [f for f in files if os.path.isdir(os.path.join(path, f))]
        dirs = [f for f in files if f != ".ipynb_checkpoints"]
        dirs = sorted(dirs)
        save_datasets_dropdown.options = dirs
    except:
        write_log(path + "が存在していませんs。")
        save_datasets_dropdown.options = ['']

def change_save_dataset(arg):
    global dataset,datasets,score_widgets,CATEGORIES
    write_log(f"change_save_dataset")
    dataset_name = arg['new'] if isinstance(arg, dict) else arg
    if dataset_name != "":
        dataset = datasets[dataset_name]
        for i, score_widget in enumerate(score_widgets):
            score_widget.children[1].value = dataset.get_count(CATEGORIES[i])

try:
    change_save_dataset(SAVE_DATASETS[0])
except:
    change_save_dataset("")
    
save_datasets_dropdown.observe(change_save_dataset, names='value')

change_save_task(SAVE_TASK[0])
save_task_dropdown.observe(change_save_task, names='value')


save_model_button.on_click(save_model)
dataset_create_button.on_click(create_dataset)
update_image_button.on_click(update_image)
change_load_task(LOAD_TASK[0])        
load_task_dropdown.observe(change_load_task, names='value')
prev_image_button.on_click(prev_image)
next_image_button.on_click(next_image)
update_image_button.on_click(update_image)
load_image_button.on_click(load_dataset)
#delete_image_button.on_click(delete_image)

In [ ]:
from os.path import join
import subprocess
import datetime
import glob
import torch.nn.functional as F
from utils import preprocess
import threading
import time

def live():
    global running, skip, sleep_time
    load_flag = True
    no = no_widget.value
    
    while running:
        no += skip
        no_widget.value = no
        try:
            load_image(no)
        except:
            write_log("no: " + no + "のファイルの読込に失敗")
        time.sleep(sleep_time/1000)

def play(c):
    global running, execute_thread, skip, sleep_time, play_num
    skip = skip_dropdown.value
    sleep_time = sleep_dropdown.value
    play_num = 0
    running = True
    execute_thread = threading.Thread(target=live)
    execute_thread.start()
    
def stop(c):
    global running, execute_thread, load_flag
    running = False
    load_flag = True
    try:
        execute_thread.join()
    except:
        write_log("現在再生されていません。")

def load_image(no):
    global last_no, img, current_path, running, play_num
    load_task_value = load_task_dropdown.value
    dataset_value = load_datasets_dropdown.value
    
    xy_path = os.path.join(current_path, load_task_value, dataset_value, "xy")
    speed_path = os.path.join(current_path, load_task_value, dataset_value, "speed")

    xy_image_names = get_file_names(xy_path)
    
    last_no = len(xy_image_names)
    if no >= last_no:
        no_widget.value = no - 1
        write_log("ファイルが存在しません。" + str(len(xy_image_names)-1) + "以内の値を設定してください。")
        running = False
        return
        
    xy_name = xy_image_names[no]
    
    img = cv2.imread(xy_name)
    preprocessed = preprocess(img)
    output = model(preprocessed)
    output = F.softmax(output, dim=1).detach().cpu().numpy().flatten()
    category_index = output.argmax()
    #prediction_widget.value = SAVE_DATATSES[category_index]
    for i, score in enumerate(list(output)):
        score_widgets[i].children[0].value = score
    next_image_button.disabled = False
    prev_image_button.disabled = False
    picture_widget.value = bgr8_to_jpeg(img)
    play_num += 1
    if play_num % 10 == 0:
        write_log(str(play_num) + "枚目を読込ました。")
    
play_button.on_click(play)
stop_button.on_click(stop)

In [ ]:
BATCH_SIZE = 8

epochs_widget = widgets.IntText(description='epochs', value=1)
eval_button = widgets.Button(description='evaluate')
train_button = widgets.Button(description='train')
loss_widget = widgets.FloatText(description='loss')
accuracy_widget = widgets.FloatText(description='accuracy')
progress_widget = widgets.FloatProgress(min=0.0, max=1.0, description='progress')

def train_eval(is_training):
    global BATCH_SIZE, LEARNING_RATE, MOMENTUM, model, device, dataset, optimizer, eval_button, train_button, accuracy_widget, loss_widget, progress_widget, state_widget
    
    try:
        optimizer = torch.optim.Adam(model.parameters())
        # optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)

        
        dataset = ImageClassificationDataset(save_task_dropdown.value + '/' + save_datasets_dropdown.value, CATEGORIES, TRANSFORMS)
            
        train_loader = torch.utils.data.DataLoader(
            dataset,
            batch_size=BATCH_SIZE,
            shuffle=True
        )
        write_log("train_loader")

        #state_widget.value = 'stop'
        train_button.disabled = True
        eval_button.disabled = True
        time.sleep(1)

        if is_training:
            model = model.train()
        else:
            model = model.eval()
        while epochs_widget.value > 0:
            i = 0
            sum_loss = 0.0
            error_count = 0.0
            for images, labels in iter(train_loader):
                # send data to device
                images = images.to(device)
                labels = labels.to(device)

                if is_training:
                    # zero gradients of parameters
                    optimizer.zero_grad()

                # execute model to get outputs
                outputs = model(images)

                # compute loss
                loss = F.cross_entropy(outputs, labels)

                if is_training:
                    # run backpropogation to accumulate gradients
                    loss.backward()

                    # step optimizer to adjust parameters
                    optimizer.step()

                # increment progress
                error_count += len(torch.nonzero(outputs.argmax(1) - labels).flatten())
                count = len(labels.flatten())
                i += count
                sum_loss += float(loss)
                progress_widget.value = i / len(dataset)
                loss_widget.value = sum_loss / i
                accuracy_widget.value = 1.0 - error_count / i
                
            if is_training:
                epochs_widget.value = epochs_widget.value - 1
            else:
                break
    except e:
        pass
    model = model.eval()

    train_button.disabled = False
    eval_button.disabled = False
    #state_widget.value = 'live'
    
train_button.on_click(lambda c: train_eval(is_training=True))
eval_button.on_click(lambda c: train_eval(is_training=False))
    
train_eval_widget = widgets.VBox([
    epochs_widget,
    progress_widget,
    loss_widget,
    accuracy_widget,
    widgets.HBox([train_button, eval_button]),
    widgets.HBox([save_model_name_widget,save_model_button])
])

In [ ]:
SKIP = [1,2,3,4,5]

movie_button = widgets.Button(description='動画の作成')
movie_name_widget = widgets.Text(description='動画名',value="run_detect_video")
movie_skip_dropdown = widgets.Dropdown(options=SKIP, description='skip(枚)', index=1)

def make_movie(change):
    global model,current_path
    
    if not movie_name_widget.value.strip():
        write_log("ファイル名を指定してください。")
        return 
    write_log("動画を作成します。")
    path = os.path.join(current_path, "video")
    if not os.path.exists(path):
        subprocess.call(['mkdir', '-p', path])
    output = path + "/" + movie_name_widget.value + ".mp4"
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = int(30/movie_skip_dropdown.value)
    outfh = cv2.VideoWriter(output, fourcc, fps, (224, 224))
    file_list = sorted(
        glob.glob(load_task_dropdown.value + '/' + load_datasets_dropdown.value + '/xy/*.jpg'),
        key=os.path.getmtime
    )
    
    xy_path = os.path.join(current_path, load_task_dropdown.value, load_datasets_dropdown.value, "xy")     
    file_list = os.listdir(xy_path)
    file_list = [os.path.join(xy_path, file_name) for file_name in file_list if file_name.endswith('.jpg')]    
    file_list = sorted(file_list, key=lambda f: extract_numbers(os.path.basename(f)))
    
    
    try:
        res_num = len(file_list)
        skip_movie = movie_skip_dropdown.value
        terminal_time = 1/(30/skip_movie)
        current_time = 0
        process_time = 0
        total_process_time = 0
        for i, file_name in enumerate(file_list):
            if i % skip_movie == 0:
                current_time += terminal_time
                img = cv2.imread(file_name)

                process_time = time.time()
                preprocessed = preprocess(img)
                output = model(preprocessed)
                output = F.softmax(output, dim=1).detach().cpu().numpy().flatten()
                category_index = output.argmax()

                thickness = 2 # テキストの太さ
                font_scale = 0.5  # フォントの大きさ
                total_process_time += time.time() - process_time 
        
                for h, score in enumerate(list(output)):
                    if h == category_index:
                        img = cv2.putText(img, f"{CATEGORIES[h]} : {int(score * 100)} %", (10, h * 15 + 40), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (0, 0, 255), thickness)
                    else:
                        img = cv2.putText(img, f"{CATEGORIES[h]} : {int(score * 100)} %", (10, h * 15 + 40), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (255, 255, 255), thickness)

                if i%(skip_movie*10) == 0:
                    write_log(f"{current_time:.1f}秒まで完了, 推論平均: {total_process_time/10*1000:.1f}ms, {int(i/skip_movie)}枚目/{int(res_num/skip_movie)}枚中を処理中")
                    total_process_time = 0
                outfh.write(img)
                time.sleep(5/1000)
                del img
    except Exception as e:
        write_log(f"error{e}")
    finally:
        # エラーが発生しても確実にリソースを解放する
        outfh.release()
        write_log("動画の出力が完了しました。")
        #get_jetson_nano_memory_usage()

movie_button.on_click(make_movie)

In [ ]:
separator = widgets.HTML('<hr style="border-color:gray;margin:10px 0"/>')
title1 = widgets.HTML('<b>■ 1.モデル読込み</b><br>分類のモデルを読込みます。')
title2 = widgets.HTML('<b>■ 2.データセット読込み</b><br>指定したデータセットを読込みます。')
title3 = widgets.HTML('<b>■ 3.データセット保存</b><br>指定したデータセットに保存します。')
title4 = widgets.HTML('<b>■ 4.アノテーション</b><br>アノテーションを実施。')
title5 = widgets.HTML('<b>■ 5.学習</b><br>転移学習を実施。')
title6 = widgets.HTML('<b>■ 6.評価動画</b><br>評価動画を作成し、目視確認します。')

data_collection_widget = widgets.VBox([
    separator,
    title1,
    widgets.HBox([load_model_widget,load_model_time_widget,load_model_button]),
    process_widget,
    separator,
    title2,
    widgets.HBox([load_datasets_dropdown, load_task_dropdown, load_image_button]),
    process_widget,
    separator,
    title3,
    widgets.HBox([save_datasets_dropdown, save_task_dropdown]),
    widgets.HBox([widgets.Label('datasetの新規作成'),dataset_name_widget,dataset_create_button]),
    process_widget,
    separator,
    title4,
    widgets.HBox([play_button,stop_button,prev_image_button,next_image_button]),
    widgets.HBox([sleep_dropdown,skip_dropdown]),
    widgets.HBox([no_widget,update_image_button,delete_image_button]),    
    widgets.HBox([picture_widget,widgets.HBox(score_widgets)]),
    process_widget,
    separator,
    title5,
    train_eval_widget,
    process_widget,
    separator,
    title6,
    widgets.HBox([movie_name_widget,movie_skip_dropdown,movie_button]),
    process_widget,
])

display(data_collection_widget)